In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import csv
import sklearn
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import accuracy_score
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization, Input, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping

2024-04-24 11:10:54.739536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 11:10:54.739589: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 11:10:54.741548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 11:10:54.883351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 11:10:56.133974: W tensorflow/compiler/

In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
train_df.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90615 entries, 0 to 90614
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              90615 non-null  int64  
 1   Sex             90615 non-null  object 
 2   Length          90615 non-null  float64
 3   Diameter        90615 non-null  float64
 4   Height          90615 non-null  float64
 5   Whole weight    90615 non-null  float64
 6   Whole weight.1  90615 non-null  float64
 7   Whole weight.2  90615 non-null  float64
 8   Shell weight    90615 non-null  float64
 9   Rings           90615 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 6.9+ MB


In [7]:
train_df["Sex"].unique()

array(['F', 'I', 'M'], dtype=object)

In [8]:
train_df.loc[train_df["Sex"]=='M', "Sex"] = 0
train_df.loc[train_df["Sex"]=="F", "Sex"] = 1
train_df.loc[train_df["Sex"]=="I", "Sex"] =2

In [10]:
train_df.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,1,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,0,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,2,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [11]:
train_df.corr()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,1.000000,-0.002642,0.004724,0.004290,0.005916,0.005228,0.004203,0.004878,0.005887,0.000938
Sex,-0.002642,1.000000,-0.553968,-0.560731,-0.540178,-0.546160,-0.523608,-0.539767,-0.540904,-0.431258
Length,0.004724,-0.553968,1.000000,0.989732,0.916094,0.931449,0.909609,0.913134,0.911073,0.623786
Diameter,0.004290,-0.560731,0.989732,1.000000,0.919618,0.933848,0.908466,0.914668,0.917871,0.636832
Height,0.005916,-0.540178,0.916094,0.919618,1.000000,0.902344,0.861769,0.886132,0.904019,0.665772
Whole weight,0.005228,-0.546160,0.931449,0.933848,0.902344,1.000000,0.971249,0.974319,0.964201,0.617274
Whole weight.1,0.004203,-0.523608,0.909609,0.908466,0.861769,0.971249,1.000000,0.949227,0.911800,0.515067
Whole weight.2,0.004878,-0.539767,0.913134,0.914668,0.886132,0.974319,0.949227,1.000000,0.937069,0.588954
Shell weight,0.005887,-0.540904,0.911073,0.917871,0.904019,0.964201,0.911800,0.937069,1.000000,0.694766
Rings,0.000938,-0.431258,0.623786,0.636832,0.665772,0.617274,0.515067,0.588954,0.694766,1.000000
